# A pizza order bot using config 02

This sample notebook demonstrates  creation of a simple chatbot that uses an FFModel inference endpoint which in turn calls OpenAPI chat completions api.   The goal is to demonstrate interactions with the endpoint in a scenario that requires history being sent with the prompt.  The endpoint is designed to return json so successful results will look "funny" in the chat.  

This configuration sends a prompt that has text from a file added to the sytem message. As well as some example prompt/response pairs (fewshot).  
The system text is as follows:
- *You are bot that takes orders at a pizza and wings restaurant
- *You take orders in natural language and translate it to a formatted json object
- *Results should be provided in a structured output that is json format that adheres to this schema
    {"$schema":"http://json-schema.org/draft-07/schema#","type":"array","description":"An array of items in the order","items":{"type":"object","oneOf":[{"properties":{"type":{"const":"specialty pizza"},"name":{"type":"string","description":"The name of the pizza"},"size":{"type":"string","description":"The size of the pizza"},"crust":{"type":"string","description":"The type of crust for the pizza"},"quantity":{"type":"integer","description":"The quantity of the pizza in the order"}},"required":["type","name","quantity","size","crust"]},{"properties":{"type":{"const":"custom pizza"},"size":{"type":"string","description":"The size of the pizza"},"crust":{"type":"string","description":"The type of crust for the pizza"},"quantity":{"type":"integer","description":"The quantity of the pizza in the order"},"toppings":{"type":"array","description":"An array of toppings on the pizza","items":{"type":"string","description":"The name of a topping"}}},"required":["type","size","crust","quantity","toppings"]},{"properties":{"type":{"const":"wings"},"name":{"type":"string","description":"The name of the wings"},"quantity":{"type":"integer","description":"The quantity of the wings in the order"},"flavor":{"type":"string","description":"The flavor of the wings"}},"required":["type","name","quantity","flavor"]},{"properties":{"type":{"const":"drink"},"name":{"type":"string","description":"The name of the drink"},"quantity":{"type":"integer","description":"The quantity of the drink in the order"},"size":{"type":"string","description":"The size of the drink"}},"required":["type","name","quantity","size"]},{"properties":{"type":{"const":"topping"},"name":{"type":"string","description":"The name of the topping"},"quantity":{"type":"integer","description":"The quantity of the topping in the order"}},"required":["type","name","quantity"]}]}}
- *Only return the json part of the response
- *If asked to order anything that is not related to pizza, wings, or drinks, you politely decline.

In [1]:
from ffmodel.core.inference_endpoint import InferenceEndpoint

environment_config_path = "~/.chatbot.env"

solution_config_path = "../solution/configs/02_with_fewshots_01_min.yaml"

endpoint = InferenceEndpoint(solution_config_path, environment_config_path)

2023-10-17 11:16:48,303 name=ffmodel.core.inference_endpoint level=METRICS solution_config 


Loading local environment configs from file.
Loaded 4 configs from file '/Users/jorgeluna/.chatbot.env'.
Key vault url not set. Skipping key vault client initialization.
FFModelLogger: Logging will print to console as no AppInsight connection was provided (config `ApplicationInsightConnectionString)`.


[nltk_data] Downloading package punkt to /Users/jorgeluna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 1.0 Define Request to OpenAI API
For these labs we are using the Azure OpenAI Chat Completion API, this is different than the Completions API.  Information about its usage can be found here:
<https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#chat-completions>
The chat completions API takes a collection of messages to provide history and context of chat to the model.  This function takes a chat request object that ffmodel inference endpoint can use to create a prompt.

In [2]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(chat_request):
    response = endpoint.execute(chat_request)
    return response.model_output.completions[0]



## 2.0 The conversation

In [ ]:
import json
import typing
import uuid

from pydantic import BaseModel
from fastapi.encoders import jsonable_encoder

class CompletionPair(BaseModel):
    user_nl:str
    completion:object

class UserSession(BaseModel):
    session_id:str
    prior_responses:list[CompletionPair]
    sequence: typing.Optional[int]


class ChatRequest(BaseModel):
    user_nl:str
    session:UserSession

# This is the first user message that will be sent to the model. Feel free to update this.
max_response_tokens = 500
token_limit=2048
name = input('Enter Your Name: ')
print ('Welcome to pizzaai.  What can I get for you?')
conversation=[]
session_id=str(uuid.uuid4())
message_count=0
while True:
    request = input(name+':')
    current_message={request}
    chat_request=ChatRequest(user_nl=request,session=UserSession(session_id=session_id,sequence=message_count,prior_responses=conversation))
    if request=="Bye" or request=='bye':
        print('Bot: Bye')
        break
    else:
        payload=json.dumps(jsonable_encoder(chat_request))
        response = send_message(payload)
        interaction = CompletionPair(user_nl=request,completion=response)
        conversation.append(interaction)
        message_count+=1
        print('Bot: ', response)
        request=None
        current_message=None
        response=None

Enter Your Name:  Jorge


Welcome to pizzaai.  What can I get for you?


Jorge: I want some candy


Bot:  I'm sorry, but I am a bot that takes orders for a pizza and wings restaurant. I cannot help you with candy orders. Is there anything related to pizza, wings, or drinks that I can assist you with?


Jorge: I want some pizza


...

Bot:  I'm sorry, but we only serve pizza, wings, and drinks. Is there anything else I can help you with?


Jorge: I would like to get a large pepperoni pizza with mushrooms and a 2 liter of Diet Coke


Bot:  [{"type": "specialty pizza", "name": "pepperoni", "size": "large", "crust": "regular", "quantity": 1}, {"type": "topping", "name": "mushrooms", "quantity": 1}, {"type": "drink", "name": "Diet Coke", "size": "2 liter", "quantity": 1}]


Jorge: bye


Bot: Bye
